In [1]:
import mindspore
import mindnlp
import numpy as np
from mindspore import context, Tensor
from mindnlp.dataset import load_dataset
dataset = load_dataset("food101", split="train[:5000]")

def show_dataset_info(dataset):
    print("dataset column: {}".format(dataset.get_col_names()))
    print("dataset size: {}".format(dataset.get_dataset_size()))
    print("dataset batch size: {}".format(dataset.get_batch_size()))
show_dataset_info(dataset)

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

dataset column: ['image', 'label']
dataset size: 5000
dataset batch size: 1


In [2]:
train_ds, val_ds = dataset.split([0.9, 0.1])
show_dataset_info(train_ds)
show_dataset_info(val_ds)

from mindnlp.transformers import ViTImageProcessor
image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

def transform(image, label):
    # 使用图像处理器处理
    processed_output = image_processor(image, return_tensors='np')

    # 获取 'pixel_values'，移除多余的批次维度
    pixel_values = processed_output['pixel_values']
    if len(pixel_values.shape) == 4 and pixel_values.shape[0] == 1:
        pixel_values = np.squeeze(pixel_values, axis=0)  # 移除第一个批次维度，变成 (3, 224, 224)
    
    labels = np.array([label], dtype=np.int32)
    return pixel_values, labels

dataset column: ['image', 'label']
dataset size: 4500
dataset batch size: 1
dataset column: ['image', 'label']
dataset size: 500
dataset batch size: 1


In [3]:
# 处理训练集
train_ds = train_ds.map(operations=transform, input_columns=["image", "label"], output_columns=["pixel_values", "labels"])
train_ds = train_ds.batch(batch_size=16, drop_remainder=True)

# 处理验证集
val_ds = val_ds.map(operations=transform, input_columns=["image", "label"], output_columns=["pixel_values", "labels"])
val_ds = val_ds.batch(batch_size=16, drop_remainder=True)

show_dataset_info(train_ds)
show_dataset_info(val_ds)

dataset column: ['pixel_values', 'labels']
dataset size: 281
dataset batch size: 16
dataset column: ['pixel_values', 'labels']
dataset size: 31
dataset batch size: 16


In [4]:
# 定义 Food101 的类别名称列表（需完整填写101个类别）
class_names = [
    'apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare',
    'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito',
    'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake',
    'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla',
    'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder',
    'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes',
    'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict',
    'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras',
    'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice',
    'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich',
    'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup',
    'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna',
    'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_soup',
    'mussels', 'nachos', 'omelette', 'onion_rings', 'oysters',
    'pad_thai', 'paella', 'pancakes', 'panna_cotta', 'peking_duck',
    'pho', 'pizza', 'pork_chop', 'poutine', 'prime_rib',
    'pulled_pork_sandwich', 'ramen', 'ravioli', 'red_velvet_cake', 'risotto',
    'samosa', 'sashimi', 'scallops', 'seaweed_salad', 'shrimp_and_grits',
    'spaghetti_bolognese', 'spaghetti_carbonara', 'spring_rolls', 'steak', 'strawberry_shortcake',
    'sushi', 'tacos', 'takoyaki', 'tiramisu', 'tuna_tartare',
    'waffles'
]

# 创建 label2id 和 id2label 字典
label2id = {name: idx for idx, name in enumerate(class_names)}
id2label = {idx: name for idx, name in enumerate(class_names)}

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )
from mindnlp.transformers import ViTForImageClassification
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=101,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
print_trainable_parameters(model)

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 85876325 || all params: 85876325 || trainable%: 100.00


In [6]:
from mindnlp.peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

trainable params: 667493 || all params: 86543818 || trainable%: 0.77


In [8]:
from mindnlp.engine import Trainer, TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-food101",
  per_device_train_batch_size=128,
  evaluation_strategy="epoch",
  num_train_epochs=5,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=5e-3,
  save_total_limit=2,
  remove_unused_columns=False,
  load_best_model_at_end=False,
)

In [9]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
)

In [10]:
train_results = trainer.train()

  1%|          | 10/1405 [00:08<07:54,  2.94it/s] 

{'loss': 2.4978, 'learning_rate': 0.004964412811387901, 'epoch': 0.04}


  1%|▏         | 20/1405 [00:10<05:57,  3.88it/s]

{'loss': 0.4403, 'learning_rate': 0.004928825622775801, 'epoch': 0.07}


  2%|▏         | 30/1405 [00:13<05:59,  3.82it/s]

...

{'loss': 0.0054, 'learning_rate': 5.338078291814947e-05, 'epoch': 4.95}


100%|█████████▉| 1400/1405 [06:19<00:01,  4.63it/s]

{'loss': 0.0013, 'learning_rate': 1.779359430604982e-05, 'epoch': 4.98}


 94%|█████████▎| 29/31 [00:03<00:00,  7.85it/s]
                                                   
100%|██████████| 1405/1405 [06:25<00:00,  3.64it/s]

{'eval_runtime': 4.0155, 'eval_samples_per_second': 7.72, 'eval_steps_per_second': 0.996, 'epoch': 5.0}
{'train_runtime': 385.5554, 'train_samples_per_second': 58.305, 'train_steps_per_second': 3.644, 'train_loss': 0.18368320296985824, 'epoch': 5.0}


In [13]:
context.set_context(mode=context.PYNATIVE_MODE, device_target='Ascend')
lora_model.set_train(False)

# 定义准确率计算函数
def calculate_accuracy(true_labels, predicted_labels):
    assert len(true_labels) == len(predicted_labels), f"标签和预测的数量必须相同，分别为 {len(true_labels)} 和 {len(predicted_labels)}"
    correct_predictions = np.sum(true_labels == predicted_labels)
    accuracy = correct_predictions / len(true_labels)
    return accuracy

# 初始化计数器
correct = 0
total = 0

# 遍历验证集
for batch in val_ds.create_dict_iterator():
    pixel_values = batch['pixel_values']
    labels = batch['labels']
    pixel_values = Tensor(pixel_values)
    labels = Tensor(labels)
    outputs = lora_model(pixel_values)
    logits = outputs.logits  # 提取 logits
    logits = logits.asnumpy()
    predicted_labels = np.argmax(logits, axis=-1)
    true_labels = labels.asnumpy().flatten()
    correct += np.sum(predicted_labels == true_labels)
    total += len(true_labels)

# 计算并打印准确率
accuracy = correct / total
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.95
